# 기본 패키지 설치, 로드, 데이터 로드

In [26]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf


sc=pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = spark.read.csv("/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/heart_2020_cleaned.csv", header=True, inferSchema=True)

Mounted at /content/drive


# 데이터 전처리

## 중복된 값 제거하고 시작

In [27]:
# 중복된 값 제거

print("원본 데이터 개수: ({}, {})".format(df.count(), len(df.columns)))

df=df.distinct()

print("중복된 행 제거 후 데이터 개수: ({}, {})".format(df.count(), len(df.columns)))

원본 데이터 개수: (301717, 18)
중복된 행 제거 후 데이터 개수: (301717, 18)


In [28]:
df.show(5)

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+
|          No| 27.2|    Yes|             No|   Yes|           0.0|         0.0|         No|  Male|80 or older|White|                  No|              No|Very good|      8.0|    No|           No|       Yes|
|          No|20.94|     No|             No|    No|          30.0|         0.0|         No|Female|      75-79|White|                  No|              No|     Poor|      6.

In [29]:
# 스키마 정보 확인
df.printSchema()

root
 |-- HeartDisease: string (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- AlcoholDrinking: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- PhysicalHealth: double (nullable = true)
 |-- MentalHealth: double (nullable = true)
 |-- DiffWalking: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Diabetic: string (nullable = true)
 |-- PhysicalActivity: string (nullable = true)
 |-- GenHealth: string (nullable = true)
 |-- SleepTime: double (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- KidneyDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)



## 데이터 인코딩, 이상치 처리, 스케일링

In [30]:
major_df = df.filter(df.HeartDisease == "No")
minor_df = df.filter(df.HeartDisease == "Yes")

# 데이터 비율 및 수
print("데이터 label 0인 경우 : {}".format(major_df.count()))
print("데이터 label 1인 경우 : {}".format(minor_df .count()))


데이터 label 0인 경우 : 274456
데이터 label 1인 경우 : 27261


In [31]:
# 컬럼 정리
str_columns = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
num_columns = ['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']


### Label 인코딩 (기본적인 Yes, no 경우의 categorical 변수는 0, 1로 인코딩)

In [32]:
# label 인코딩
from pyspark.ml.feature import StringIndexer 

indexed = df
for col in str_columns:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_Index")
    indexed = indexer.fit(indexed).transform(indexed)
indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------

### Ordinal Encoding (나이, 건강상태, 당뇨병 강도는 순서가 있다고 판단 Ordinal 인코딩 진행)

In [33]:
# ordinal encoding

from pyspark.sql.functions import when
# ord_columns=['AgeCategory', 'Diabetic', 'GenHealth']
# age_map = {'18-24' : 20, '25-29' : 25, '30-34' : 30, '35-39' : 35, '40-44' : 40, '45-49' : 45, '50-54' : 50, '55-59' : 55, '60-64' : 60, '65-69' : 65, '70-74' : 70, '75-79' : 75, '80 or older' : 80}
# genhealth_map = {'Excellent' : 4, 'Very good' : 3, 'Good' : 2, 'Fair' : 1, 'Poor' : 0}
# diabetic_map={'Yes' : 2, 'Yes (during pregnancy)' : 2, 'No, borderline diabetes' : 1, 'No' : 0}

# Age
indexed=indexed.withColumn("AgeCategory_Index", when(indexed.AgeCategory=="18-24", 20.0)
  .when(indexed.AgeCategory=="25-29", 25.0)
  .when(indexed.AgeCategory=="30-34", 30.0)
  .when(indexed.AgeCategory=="35-39", 35.0)
  .when(indexed.AgeCategory=="40-44", 40.0)
  .when(indexed.AgeCategory=="45-49", 45.0)
  .when(indexed.AgeCategory=="50-54", 50.0)
  .when(indexed.AgeCategory=="55-59", 55.0)
  .when(indexed.AgeCategory=="60-64", 60.0)
  .when(indexed.AgeCategory=="65-69", 65.0)
  .when(indexed.AgeCategory=="70-74", 70.0)
  .when(indexed.AgeCategory=="75-79", 75.0)
  .otherwise(80.0))

# GenHealth
indexed=indexed.withColumn("GenHealth_Index", when(indexed.GenHealth=="Excellent", 4.0)
  .when(indexed.GenHealth=="Very good", 3.0)
  .when(indexed.GenHealth=="Good", 2.0)
  .when(indexed.GenHealth=="Fair", 1.0)
  .otherwise(0.0))

# Diabetic 2, 2, 1, 0으로 설정
indexed=indexed.withColumn("Diabetic_Index", when(indexed.Diabetic=="Yes", 2.0)
  .when(indexed.Diabetic=="Yes (during pregnancy)", 2.0)
  .when(indexed.Diabetic=="No, borderline diabetes", 1.0)
  .otherwise(0.0))

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|
+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+-------------

### Race 원핫 인코딩

In [34]:
# Race onehot 인코딩 -> 미미해서 feature 제거할 수도 있음
from pyspark.ml.feature import OneHotEncoder

indexer = StringIndexer(inputCols=['Race'], outputCols=['Race_Index(temp)'])
indexed = indexer.fit(indexed).transform(indexed)

ohe_encoder = OneHotEncoder(inputCols=['Race_Index(temp)'], outputCols=['Race_Index'], dropLast=True)
indexed = ohe_encoder.fit(indexed).transform(indexed)

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|
+------------+-----+-------+---------------+------+--------------+------------+---------

### 이상치 제거 및 변환

In [35]:
# BMI 80 이상과, 수면시간이 24시간은 말이 안되므로 제거

indexed=indexed.filter((indexed.BMI < 80) & (indexed.SleepTime != 24))

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|
+------------+-----+-------+---------------+------+--------------+------------+---------

In [36]:
# 도메인 지식 없이 데이터 단순 지우기는 버겁다 생각-> log transformation 이용해서 skew값 안정화


# BMI
indexed=indexed.withColumn("BMI_Log", F.log1p("BMI"))

# PhysicalHealth
indexed=indexed.withColumn("PhysicalHealth_Log", F.log1p("PhysicalHealth"))

# MentalHealth
indexed=indexed.withColumn("MentalHealth_Log", F.log1p("MentalHealth"))

# SleepTime
indexed=indexed.withColumn("SleepTime_Log", F.log1p("SleepTime"))

indexed.show()


+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+------------------+------------------+------------------+------------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Race_Index(temp)|   Race_Index|           BM

In [37]:
# 이상치 제거 후 비율 재확인
major_df = indexed.filter(indexed.HeartDisease_Index == 0.0)
minor_df = indexed.filter(indexed.HeartDisease_Index == 1.0)

print("이상치 제거 후 데이터 개수: ({}, {})".format(indexed.count(), len(indexed.columns)))
print("데이터 label 0인 경우 : {}".format(major_df.count()))
print("데이터 label 1인 경우 : {}".format(minor_df.count()))

이상치 제거 후 데이터 개수: (301653, 37)
데이터 label 0인 경우 : 274401
데이터 label 1인 경우 : 27252


### 수치형 변수 standardScaler를 통한 스케일링

In [38]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

num_columns = ['BMI_Log', 'PhysicalHealth_Log', 'MentalHealth_Log', 'SleepTime_Log']

assembler = VectorAssembler(inputCols=num_columns, outputCol="num_feauture")
num_vec = assembler.transform(indexed)
scaler = StandardScaler(inputCol="num_feauture", outputCol="num_feauture_Index", withMean=True, withStd=True)  # True로 하면 평균은 0, std=1로 '표준정규분포'
indexed= scaler.fit(num_vec).transform(num_vec)

indexed.show()

+------------+-----+-------+---------------+------+--------------+------------+-----------+------+-----------+-----+--------------------+----------------+---------+---------+------+-------------+----------+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+----------------+-------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|HeartDisease|  BMI|Smoking|AlcoholDrinking|Stroke|PhysicalHealth|MentalHealth|DiffWalking|   Sex|AgeCategory| Race|            Diabetic|PhysicalActivity|GenHealth|SleepTime|Asthma|KidneyDisease|SkinCancer|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|Ra

### 최종 이용 컬럼 선택

In [39]:
# 최종 사용 컬럼
label=indexed.columns[18]
use_variable=indexed.columns[19:]
use_variable.remove('Race_Index(temp)')
use_variable.remove('Race_Index') 
use_variable.remove('num_feauture')
use_variable.remove('num_feauture_Index')


final_col=indexed.columns[18:]
final_col.remove('Race_Index(temp)')
final_col.remove('Race_Index')
final_col.remove('num_feauture')
final_col.remove('num_feauture_Index')


final_col

['HeartDisease_Index',
 'Smoking_Index',
 'AlcoholDrinking_Index',
 'Stroke_Index',
 'DiffWalking_Index',
 'Sex_Index',
 'PhysicalActivity_Index',
 'Asthma_Index',
 'KidneyDisease_Index',
 'SkinCancer_Index',
 'AgeCategory_Index',
 'GenHealth_Index',
 'Diabetic_Index',
 'BMI_Log',
 'PhysicalHealth_Log',
 'MentalHealth_Log',
 'SleepTime_Log']

In [40]:
# 기본적인 최종 데이터 추출

final_data=indexed.select(final_col)

final_data.show()

+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|               0.0|          1.0|                  0.0|         1.0|              0.0|      1.0|               

#데이터 분리 (데이터 증강 및 삭제는 반드시 분리된 상태의 원본 train_data를 기준으로만 진행해야 함.)

In [41]:
# 원본 데이터 분리 -기준점

train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

train_data.show()

+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|               0.0|          0.0|                  0.0|         0.0|              0.0|      0.0|               

In [42]:
# Cost-Sensitive Learning을 위해 가중치 추가한 파일 -> 해당 cost_train_data는 비용민감 학습에만 이용할 것 가중치 열은 train set에만 추가

cost_train_data=train_data

# weight 열 추가
cost_train_data=cost_train_data.withColumn("weight", when(cost_train_data.HeartDisease_Index==1.0, 0.91)
  .otherwise(0.09))

cost_train_data.show()

+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|weight|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+------+
|               0.0|          0.0|                  0.0|         0.0|              0.0|    

# 기본 전처리 완료된 파일 내보내기

In [43]:
train_data.toPandas().to_csv('/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/train_data.csv', index=False, encoding='CP949')
test_data.toPandas().to_csv('/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/test_data.csv', index=False, encoding='CP949')
cost_train_data.toPandas().to_csv('/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/cost_train_data.csv', index=False, encoding='CP949')